#### Домашнее задание к занятию "Проблема качества данных"

Необходимо запустить практическую часть занятия, и посмотреть самому то, о чём говорили на лекции. По образу практики, попробуйте создать искусственный датасет с лишними столбцами. Целевую метку, при правильной обработке данных, формируйте таким образом, чтобы без затруднений её смогла описать линейная модель. Ориентируйтесь на то, что было показано во время занятия, и каждый шаг описывайте в markdown. Здесь важно видеть ваш ход мысли. Не бойтесь ошибиться или написать не то. Данное задание не имеет какого-то “правильного” решения. Цель - достичь базового понимания проблемы. Чем больше вы фантазируете, тем лучше :) Тем не менее, старайтесь представить те ситуации, которые по-вашему мнению могли бы быть в реальных данных. Успеха!

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

Создадим датасет с данными о продажах продукции по различным компаниям и рыночной стоимостью этих компаний.
Если упростить один из методов оценки стоимости компании, то стоимость складывается из
1. имеющихся активов (предположим, что все компании располагают активами в $200200),
2. прогноза будущих доходов за какой-то период времени (возьмем 10 лет) за вычетом всех расходов на поддержание бизнеса, налогов и прочих обязательств (будем считать, что на все это уходит 80% от дохода)

In [2]:
# Создаем сэмпл
n_samples = 500

# количество персонала в компании
number_of_stuff = np.random.choice(500, n_samples) + 10

# количество проданных единиц товара за год
quantity_of_sold_goods  = np.random.choice(100100, n_samples) + 1000

# средняя цена единицы товара
average_price_of_sold_goods  = np.random.choice(100, n_samples) + 10

# стоимость компании
company_value = quantity_of_sold_goods * average_price_of_sold_goods * 10 * 0.2 + 200200

data = pd.DataFrame({'number_of_stuff': number_of_stuff, 'quantity_of_sold_goods': quantity_of_sold_goods, 
                     'average_price_of_sold_goods': average_price_of_sold_goods, 'company_value': company_value})
data.head(5)

,number_of_stuff,quantity_of_sold_goods,average_price_of_sold_goods,company_value
0,481,90488,34,6353384.0
1,422,46396,21,2148832.0
2,357,87939,25,4597150.0
3,278,33299,13,1065974.0
4,176,28547,58,3511652.0


Обучаем модель линейной регрессии и смотрим на получившиеся веса, сдвиг и среднюю абсолютную ошибку

In [3]:
from sklearn.metrics import mean_absolute_error

# Определяем параметры
X = data[['number_of_stuff', 'quantity_of_sold_goods', 'average_price_of_sold_goods']]

# Определяем целевое значение
y = data['company_value']

# Обучаем модель
reg = LinearRegression().fit(X, y)

# Определяем веса (коэффициенты перед Х):
print('Weights: {}'.format(reg.coef_))

# Определяем сдвиг, который был равен 200200 по условию
print('Bias: {}'.format(reg.intercept_))

# С помощью обученной модели и имеющихся параметров Х предсказываем целевые значения у
pred_values = reg.predict(X)

# Считаем абсолютную ошибку
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [   496.67847665    116.69429107 101892.00740179]
Bias: -5778267.020082743
Error: 1241957.5142400449


In [4]:
y.median()

5092750.0

В результате видим достаточно сомнительные веса, неверный сдвиг и огромную среднюю абсолютную ошибку (около четверти).  
Попробуем убрать лишний признак, т.е. количество персонала, которое не влияет на рыночную капитализацию компании (строго говоря, косвенно немного влияет, но это влияние настолько невелико, что можно легко им пренебречь безо всякого ущерба для модели).

In [5]:
# Определяем параметры (уже без персонала)
X = data[['quantity_of_sold_goods', 'average_price_of_sold_goods']]

# Определяем целевое значение
y = data['company_value']

# Обучаем модель
reg = LinearRegression().fit(X, y)

# Определяем веса (коэффициенты перед Х):
print('Weights: {}'.format(reg.coef_))

# Определяем сдвиг, который был равен 200200 по условию
print('Bias: {}'.format(reg.intercept_))

# С помощью обученной модели и имеющихся параметров Х предсказываем целевые значения у
pred_values = reg.predict(X)

# Считаем абсолютную ошибку
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [   116.65491981 101901.54532945]
Bias: -5646108.2870503655
Error: 1239859.7060360666


Результаты не стали лучше.  
Чтобы понять почему, нужно ответить на вопрос: что из себя представляет линейная модель?  
f(x,b) = b0 + b1x1 + b2x2 и т.д., где b - параметры (коэффициенты) регрессии, х - регрессоры (факторы модели).  
В нашем примере получается так: b0 + b1x1x2, что не является линейной зависимостью и приводит к огромным погрешностям модели.  
Поэтому необходимо создать новый признак, чтобы получить зависимость вида: b0 + b1x1.

In [6]:
# Создаем новый признак - годовую прибыль (annual income), т.е. [кол-во проданного за год товара] * [средняя цена товара]
data['annual_income'] = data['quantity_of_sold_goods'] * data['average_price_of_sold_goods']
data.head(5)

,number_of_stuff,quantity_of_sold_goods,average_price_of_sold_goods,company_value,annual_income
0,481,90488,34,6353384.0,3076592
1,422,46396,21,2148832.0,974316
2,357,87939,25,4597150.0,2198475
3,278,33299,13,1065974.0,432887
4,176,28547,58,3511652.0,1655726


In [7]:
# Определяем параметры
X = data[['annual_income']]

# Определяем целевое значение
y = data['company_value']

# Обучаем модель
reg = LinearRegression().fit(X, y)

# Определяем веса (коэффициенты перед Х):
print('Weights: {}'.format(reg.coef_))

# Определяем сдвиг, который был равен 200200 по условию
print('Bias: {}'.format(reg.intercept_))

# С помощью обученной модели и имеющихся параметров Х предсказываем целевые значения у
pred_values = reg.predict(data[['annual_income']])

# Считаем абсолютную ошибку
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [2.]
Bias: 200200.0
Error: 0.0


Получили абсолютно верные вес (10 лет * 20% = 2) и сдвиг (200200 как в условии), а также нулевую ошибку. И все это по причине того, что мы создали линейную зависимость с помощью нового признака.